In [1]:
# 必要なモジュールをインポート
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import pandas as pd

# 環境変数の取得
load_dotenv()

# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.environ['API_KEY'])

# モデル名
MODEL_NAME = "gpt-4o-mini"

In [ ]:
# アンケートの要約をLLMで抽出する関数
def extract_summary(text):
    # 2. データをLLM用にテキスト形式に変換
    # プロンプトの作成
    prompt_text = f"以下のアンケートを10字以内に要約したタイトルをつけてください。\nアンケート: {text}"

    # 3. OpenAI APIの呼び出し
    try:
        # OpenAI APIで応答を生成
        response = client.chat.completions.create(
            model=MODEL_NAME,
            messages=[
                {"role": "user", "content": prompt_text},
            ],
            max_tokens=100,  # 応答の長さを調整
            temperature=0.3  # 創造性の調整
        )
        # 要約を取得
        summary = response.choices[0].message.content.strip()
        return summary
    
    except Exception as e:
        print(f"API呼び出しでエラーが発生しました: {e}")
        return "エラー"


In [3]:
# ワークフロー化
print("処理を開始します。")

# Excelファイルを読み込む
df = pd.read_excel('サンプルデータ.xlsx', sheet_name='アンケート')

# A列の各行のアンケートに対して要約を実行し、B列に書き込む
df['要約'] = df['アンケート'].apply(extract_summary)

# 結果をExcelファイルに保存
df.to_excel("アンケート要約結果.xlsx", index=False)

print("Excelファイルに要約結果を保存しました。")

処理を開始します。
Excelファイルに要約結果を保存しました。


In [4]:
# アンケート要約結果.xlsxに出力した内容
df

,アンケート,要約
0,「使った瞬間、髪がしっとりサラサラになり驚きました。香りも強すぎずリラックスできる香りで気に...,「しっとりサラサラ体験」
1,「洗い上がりはスッキリするのに、髪が全然きしまず指通りが良くなりました！乾燥しがちな髪質です...,「スッキリ髪、指通り良好」
2,「香りが少し甘めで、好みが分かれそうです。個人的には好きですが、甘い香りが苦手な方には少し強...,「甘めの香り評価」
3,「ボトルのデザインがシンプルでおしゃれなので、バスルームが映える感じがします。使用感も良くて...,「おしゃれでしっとり」
4,「使い始めてから頭皮のかゆみが減りました。敏感肌なので合わないシャンプーが多いのですが、これ...,「敏感肌に優しいシャンプー」
